In [1]:
import torch
from tqdm import tqdm_notebook as tqdm

In [2]:
from net import ResNet20
from train import train_epoch

Files already downloaded and verified
Files already downloaded and verified


In [3]:
res_net = ResNet20()

In [4]:
res_net = res_net.cuda()

In [5]:
criterion = torch.nn.CrossEntropyLoss()

In [6]:
test_accuracies = []

In [7]:
optimizer = torch.optim.SGD(res_net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
res_net.train()

for _ in tqdm(range(20)):
    test_accuracy = train_epoch(res_net, criterion, optimizer)
    test_accuracies.append(test_accuracy)
    print(test_accuracy)

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


0.4597999999999999
0.6062000000000001
0.6799000000000001
0.7258000000000001
0.7331
0.7616999999999998
0.7664999999999997
0.7709000000000001
0.7784999999999997
0.797
0.8074000000000001
0.7922
0.8031
0.8126000000000002
0.8181999999999999
0.812
0.8007
0.8231999999999999
0.8219999999999998
0.8259000000000001



### Уменьшаем lr в 10 раз

In [10]:
optimizer = torch.optim.SGD(res_net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
for _ in tqdm(range(200)):
    test_accuracy = train_epoch(res_net, criterion, optimizer)
    test_accuracies.append(test_accuracy)
    print(test_accuracy)

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0.8599999999999999
0.8563
0.8610000000000002
0.8606999999999999
0.8612999999999998
0.8648
0.8617
0.867
0.8662000000000001
0.8633
0.8674999999999998
0.8639999999999999
0.8687
0.8626000000000001
0.8622
0.8679000000000001
0.8641000000000001
0.8614999999999999
0.8632000000000001
0.8695
0.8633000000000002
0.8680000000000001
0.8702
0.8688999999999999
0.8669000000000002
0.8679000000000001
0.8667000000000002
0.8676999999999999
0.8682000000000002
0.8685
0.8704999999999999
0.8699999999999999
0.8725
0.8706
0.8594000000000002
0.8695
0.8716000000000002
0.8715
0.8665
0.866
0.8691000000000002
0.8665999999999999
0.871
0.8665999999999999
0.8665999999999999
0.8649000000000001
0.8706000000000002
0.8638000000000001
0.866
0.8702999999999999
0.8668999999999998
0.8643000000000001
0.8697
0.8654000000000001
0.8645999999999999
0.8637000000000001
0.8662000000000002
0.8686999999999999
0.8655999999999999
0.8645
0.8730999999999999
0.8614999999999999
0.8672
0.8703
0.8634000000000002
0.8694
0.8679000000000001
0.86920

### Уменьшаем lr в 10 раз

In [11]:
optimizer = torch.optim.SGD(res_net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
for _ in tqdm(range(200)):
    test_accuracy = train_epoch(res_net, criterion, optimizer)
    test_accuracies.append(test_accuracy)
    print(test_accuracy)

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0.8813000000000001
0.8816000000000002
0.8806999999999999
0.8808999999999999
0.8805999999999998
0.8802000000000001
0.8806999999999999
0.8789000000000001
0.8804999999999998
0.8830999999999999
0.8837999999999999
0.8843
0.883
0.8817999999999999
0.8832000000000001
0.8823000000000002
0.8830999999999999
0.8814
0.8828
0.884
0.8806000000000002
0.8842
0.8819
0.8841000000000001
0.8834000000000002
0.8819999999999999
0.8829
0.8846000000000003
0.8829
0.8835999999999999
0.8845000000000001
0.8839
0.8838000000000001
0.8835000000000001
0.8831
0.8805999999999998
0.8818
0.8840000000000002
0.8832000000000001
0.8819
0.8828999999999999
0.8830999999999999
0.8819000000000001
0.8813000000000001
0.8819000000000001
0.8817999999999999
0.8804000000000001
0.8811
0.8826999999999998
0.8817999999999999
0.8800999999999998
0.8810000000000001
0.8812000000000001
0.8816000000000002
0.8816000000000002
0.8822999999999999
0.8822
0.8810999999999999
0.8825
0.8830000000000001
0.8806
0.8811999999999999
0.8809
0.8827999999999999
0.